## Collection and Preparation:

In a MLOps approch, this jupyter notebook will Collect, Create the dataset and store it into a AWS like Bucket ( MiniIo).

Import and create the folder data:

In [17]:
! pip install pandas
! pip install tqdm
! pip install numpy
! pip install boto3
! mkdir data


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
mkdir: cannot create directory ‘data’: File exists


In [43]:
import os
import pandas as pd
import tqdm
import numpy as np
import random
import boto3
from botocore.exceptions import ClientError


In Montréal, in 2025, we have 15 skate spots. Some skatepark are greater to skate than others, for example, Ahuntsic 's skatepark have more modules than Verdun 's skate park.

Why create a model if you already know which parks is the best?
Depending on the traffic, on the day (and the occupation, because some skateparks are more busy the week end than others) the distance, we don't go to the same spot every session.

## The problem:
Ususally my friends ask me at the last minute: "Seb, where we go today?" And every time, I have no idea of which skatepark could be the best option. So that's a great problem to solve as a MLOps Software Engineer like me :) 


In [19]:
skateparks_list = [
    "Ahuntsic"         , # 0
    "VanHorne"         , # 1
    "Verdun"           , # 2
    "Lasalle"          , # 3
    "Préfontaine"      , # 4
    "Boucherville"     , # 5
    "Taz"              , # 6
    "Spin"             , # 7
    "Saint Jérome"     , # 8
    "Saint Sauveur"    , # 9
    "Assomption"       , # 10
    "Benny"            , # 11
    "Dorval"           , # 12
    "Magog"            , # 13
    "Berthierville"      # 14
]

# Weather: 
# Rain or Snow  = 0
# Clouds + Wind = 1
# Clouds        = 2
# Blue sky      = 3

weather_dict = {
    0: "Rain or Snow",
    1: "Windy or heavily cloudy",
    2: "cloudy",
    3: "Blue sky"
}

Temperature will be in °C
traffic time: in minutes

We will give numbers index for the weekday:
0: Monday -> 6: Sunday

#### For this test, we will the index of the skatepark, the weather, the temperature and the traffic time

In [4]:
N=  20
data = []

for idx in tqdm.tqdm(range(N)):
    try:
        weather = random.randint(0, 3)
        temperature = np.clip(np.random.normal(22,6), -5, 38)
        traffic_time = np.clip(np.random.normal(25, 10), 5, 150)

        print(f"\n\n météo: {weather_dict[weather]}, a {temperature}°C\t, temps de traffic: {traffic_time}min")
        for idx_spot, spot_name in enumerate(skateparks_list):
            satisfaction = input(f"{spot_name}\tSatisfaction? (0 or 1)")
            data.append({
                "spot_name": spot_name,
                "index spot": idx_spot,
                "weather": weather,
                "temperature": round(float(temperature), 1),
                "traffic_time": round(float(traffic_time), 1),
                "satisfaction": round(int(satisfaction))
            })
    except Exception as e:
        print(e)


  0%|          | 0/20 [00:00<?, ?it/s]



 météo: cloudy, a 14.03347034597965°C	, temps de traffic: 34.396434207321114min


  0%|          | 0/20 [00:18<?, ?it/s]


KeyboardInterrupt: Interrupted by user

## Data Augmentation:

I will not change my choice for:
_ temperature if the delta temp is 5°C
_ traffic time if it's between 10 min

So, I can take random variations!

In [33]:
data = []

for weather in range(4):
    for idx_spot, spot_name in enumerate(skateparks_list):
        print(weather_dict[weather])
        # Low temperature:
        temp = random.uniform(-10, 8)
        print(f"{temp}°C")
        satisfaction = input(f"{spot_name}\tSatisfaction? (0 or 1)")
        data.append({
            "spot_name": spot_name,
            "index spot": idx_spot,
            "weather": weather,
            "temperature": round(float(temp), 1),
            # "traffic_time": round(float(traffic_time), 1),
            "satisfaction": round(int(satisfaction))
        })

        # Good temperature:
        temp = random.uniform(15, 30)
        print(f"{temp}°C")
        satisfaction = input(f"{spot_name}\tSatisfaction? (0 or 1)")
        data.append({
            "spot_name": spot_name,
            "index spot": idx_spot,
            "weather": weather,
            "temperature": round(float(temp), 1),
            # "traffic_time": round(float(traffic_time), 1),
            "satisfaction": round(int(satisfaction))
        })

Rain or Snow
-6.294287920950271°C


Ahuntsic	Satisfaction? (0 or 1) 0


23.691680277303394°C


Ahuntsic	Satisfaction? (0 or 1) 0


Rain or Snow
3.962472074377505°C


...

Magog	Satisfaction? (0 or 1) 1


Blue sky
4.914590470861251°C


Berthierville	Satisfaction? (0 or 1) 0


16.429900337102595°C


Berthierville	Satisfaction? (0 or 1) 1


In [44]:
augmented_data = []

index = 0
while index < len(data):
    current_data = data[index]
    for i in range (20):
        new_temp = current_data["temperature"] + random.uniform(-5, 5)  # Alpha temp = 3°C
        augmented_data.append({
            "spot_name": current_data["spot_name"],
            "index spot": current_data["index spot"],
            "weather": current_data["weather"],
            "temperature": new_temp,
            # "traffic_time": new_traffic_time,
            "satisfaction": current_data["satisfaction"]
        })

    index += 1

data += augmented_data

In [35]:

df = pd.DataFrame(data)
print(df.head(10))

df.to_csv("data/skatepark_dataset_cleanup.csv", index=False)
print("\nDone")    

     spot_name  index spot  weather  temperature  satisfaction
0     Ahuntsic           0        0         -6.3             0
1     Ahuntsic           0        0         23.7             0
2     VanHorne           1        0          4.0             0
3     VanHorne           1        0         24.3             1
4       Verdun           2        0          7.2             0
5       Verdun           2        0         18.8             0
6      Lasalle           3        0         -2.0             0
7      Lasalle           3        0         18.1             0
8  Préfontaine           4        0          3.2             0
9  Préfontaine           4        0         21.4             0

Done


## AWS like project:

We will use minIo, to use a local bucket like AWS S3. The goal is to be able to switch on AWS if in the future the project become more serious.
MinIO run In another Docker container, to have access to the GUI:

http://127.0.0.1:9001/login


In [45]:
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
S3_ENDPOINT = os.getenv("S3_ENDPOINT")

In [46]:
s3 = boto3.client(
    "s3",
    endpoint_url = S3_ENDPOINT,
    aws_access_key_id = AWS_ACCESS_KEY_ID,
    aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
    region_name="us-east-1"    
)

BUCKET_NAME = "skatedata"
FILE_PATH = "data/skatepark_dataset_cleanup.csv"
#FILE_PATH = "data/skatepark_dataset_test.csv"
OBJECT_NAME = "clean_skatepark_dataset.csv"

try:
    s3.create_bucket(Bucket = BUCKET_NAME)
    print(f"[+] {BUCKET_NAME} bucket created!")
except ClientError as e:
    if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
        print(f"[i] Bucket '{BUCKET_NAME}' already exists...")
    else:
        print(f"[-]{e}")

[i] Bucket 'skatedata' already exists...


In [41]:
s3.upload_file(FILE_PATH, BUCKET_NAME, OBJECT_NAME)

Check if the csv is inside the bucket

In [50]:
print("[i] file inside the bucket:")
for obj in s3.list_objects_v2(Bucket=BUCKET_NAME).get("Contents", []):
    print(f" - {obj['Key']}")

[i] file inside the bucket:
 - clean_skatepark_dataset.csv


In [48]:
# To delete files from buckets:

keys_to_delete = {
    "Objects": [
        {"Key": "clean_skatepark_dataset.csv"},
        {"Key": "cleaned_skatepark_dataset.csv"},
    ]
}

s3.delete_objects(Bucket="skatedata", Delete=keys_to_delete)


{'ResponseMetadata': {'RequestId': '187D8FA448AE9472',
  'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '232',
   'content-type': 'application/xml',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
   'x-amz-request-id': '187D8FA448AE9472',
   'x-content-type-options': 'nosniff',
   'x-ratelimit-limit': '2060',
   'x-ratelimit-remaining': '2060',
   'x-xss-protection': '1; mode=block',
   'date': 'Wed, 03 Dec 2025 01:16:04 GMT'},
  'RetryAttempts': 0},
 'Deleted': [{'Key': 'clean_skatepark_dataset.csv'},
  {'Key': 'cleaned_skatepark_dataset.csv'}]}